In [4]:
# import pandas as pd
import tkinter as tk
from tkinter import ttk
# from PIL import Image, ImageOps
# from pillow_heif import register_heif_opener
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from lxml import etree
from docx.enum.table import WD_TABLE_ALIGNMENT
import traceback
from docx.enum.text import WD_ALIGN_PARAGRAPH
import pandas as pd
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

def set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000"):
    p = paragraph._element
    pPr = p.find(qn('w:pPr'))
    if pPr is None:
        pPr = OxmlElement('w:pPr')
        p.append(pPr)

    shd = OxmlElement('w:shd')
    shd.set(qn('w:val'), 'clear')
    shd.set(qn('w:color'), 'auto')
    shd.set(qn('w:fill'), fill_color)
    pPr.append(shd)

    for border_type in ['top', 'bottom', 'left', 'right']:
        border = OxmlElement(f'w:{border_type}')
        border.set(qn('w:val'), 'single')
        border.set(qn('w:sz'), '4')
        border.set(qn('w:space'), '0')
        border.set(qn('w:color'), border_color)
        pPr.append(border)

def set_cell_paragraph_formatting(paragraph):
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_before = Pt(6)

def set_column_width(cell, width):
    cell.width = Inches(width)
    tcPr = cell._element.get_or_add_tcPr()
    tcW = OxmlElement('w:tcW')
    tcW.set(qn('w:w'), str(int(width * 1440)))  # width in twips (1 inch = 1440 twips)
    tcW.set(qn('w:type'), 'dxa')
    tcPr.append(tcW)

doc = Document('Base Template.docx')

# Remove any existing paragraphs if they exist (this part is crucial)
if len(doc.paragraphs) > 0:
    for para in doc.paragraphs:
        if not para.text.strip():  # Check if the paragraph is empty
            p = para._element
            p.getparent().remove(p)  # Remove any empty paragraphs

# Adding a paragraph with specific formatting
paragraph = doc.add_paragraph()
run = paragraph.add_run("REGIONAL DATA")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Ensure no extra space before the paragraph
paragraph_format = paragraph.paragraph_format
paragraph_format.space_before = Pt(0)  # Set space before to 0
paragraph_format.space_after = Pt(0)   # Set space after to 0

# Set the background and border
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

# Add the PRG Logo image to the same paragraph
paragraph = doc.add_paragraph()
paragraph.alignment = 1  # Center alignment

# Add the PRG Logo image to the paragraph
paragraph.add_run().add_picture('frederick_location_chart.jpg', width=Inches(2.0))  # Adjust width as needed

# Save the document
doc.save('location_chart_document.docx')